<a href="https://colab.research.google.com/github/YassGan/RTG-SLAM/blob/main/SPLATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt update
!apt install python3.10 python3.10-distutils
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10
!python3.10 --version  # Should show 3.10.x

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,767 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,692 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packa

In [2]:
!python3 --version

Python 3.11.11


In [3]:
!sudo update-alternatives --remove-all python3  # Clear existing alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --config python3  # If prompted, select 3.10
!python3 --version

update-alternatives: using /usr/bin/python3.10 to provide /usr/bin/python3 (python3) in auto mode
There is only one alternative in link group python3 (providing /usr/bin/python3): /usr/bin/python3.10
Nothing to configure.
Python 3.10.12


In [4]:
!python3 --version

Python 3.10.12


In [8]:
# Install CUDA 11.6 toolkit (full, not just runtime)
!apt install -y cuda-toolkit-11-6
!export PATH=/usr/local/cuda-11.6/bin:$PATH
!export LD_LIBRARY_PATH=/usr/local/cuda-11.6/lib64:$LD_LIBRARY_PATH
!nvcc --version  # Should show 11.6

# Install compatible GCC (e.g., gcc-9)
!apt install -y gcc-9 g++-9
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-9 90
!update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-9 90
!gcc --version  # Should show 9.x!python3.10 -c "import sys; print(sys.version)"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package cuda-toolkit-11-6
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Fri_Dec_17_18:16:03_PST_2021
Cuda compilation tools, release 11.6, V11.6.55
Build cuda_11.6.r11.6/compiler.30794723_0
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  cpp-9 gcc-9-base libasan5 libgcc-9-dev libstdc++-9-dev
Suggested packages:
  gcc-9-locales g++-9-multilib gcc-9-doc gcc-9-multilib libstdc++-9-doc
The following NEW packages will be installed:
  cpp-9 g++-9 gcc-9 gcc-9-base libasan5 libgcc-9-dev libstdc++-9-dev
0 upgraded, 7 newly installed, 0 to remove and 30 not upgraded.
Need to get 41.2 MB of archives.
After this operation, 138 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 gc

In [6]:


# Install CUDA 11.6
!wget https://developer.download.nvidia.com/compute/cuda/11.6.0/local_installers/cuda_11.6.0_510.39.01_linux.run
!sudo sh cuda_11.6.0_510.39.01_linux.run --silent --toolkit --no-drm --no-man-page
!export PATH=/usr/local/cuda-11.6/bin:$PATH
!export LD_LIBRARY_PATH=/usr/local/cuda-11.6/lib64:$LD_LIBRARY_PATH
!nvcc --version  # Should show 11.6

--2025-03-23 22:56:26--  https://developer.download.nvidia.com/compute/cuda/11.6.0/local_installers/cuda_11.6.0_510.39.01_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.43.51.15, 23.43.51.10
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.43.51.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3488951771 (3.2G) [application/octet-stream]
Saving to: ‘cuda_11.6.0_510.39.01_linux.run’

cuda_11.6.0_510.39. 100%[===================>]   3.25G  82.8MB/s    in 1m 47s  

2025-03-23 22:58:13 (31.1 MB/s) - ‘cuda_11.6.0_510.39.01_linux.run’ saved [3488951771/3488951771]

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Fri_Dec_17_18:16:03_PST_2021
Cuda compilation tools, release 11.6, V11.6.55
Build cuda_11.6.r11.6/compiler.30794723_0


In [7]:
# Install PyTorch 1.12.1 with CUDA 11.6
!python3.10 -m pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 -f https://download.pytorch.org/whl/torch_stable.html

# Clone SplaTAM
!git clone https://github.com/spla-tam/SplaTAM.git
%cd SplaTAM

# Install requirements with python3.10
!python3.10 -m pip install -r requirements.txt
!python3.10 -m pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 GB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 123.5 MB/s eta 0:00:00
Cloning into 'SplaTAM'...
remote: Enumerating objects: 210, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 210 (delta 93), reused 41 (delta 41), pack-reused 79 (from 2)
Receiving objects: 100% (210/210), 14.35 MiB | 16.55 MiB/s, done.
Resolving deltas: 100% (108/108), done.
/content/SplaTAM
  Cloning https://github.com/JonathonLuiten/diff-gaussian-rasterization-w-depth.git (to revision cb65e4b86bc3bd8ed42174b72a62e8d3a3a71110) to /tmp/pip-req-build-sypp2omd
  R

In [ ]:
!python3.10 --version  # 3.10.12
!nvcc --version  # CUDA 11.6
!python3.10 -c "import torch; print(torch.__version__); print(torch.cuda.is_available())"  # 1.12.1+cu116, True